# Challenge Exercise

<!-- Challenge Exercise -->
### Using Spark SQL, create DataFrames, which filter for the following:

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import*

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

import pandas as pd

Start the Sesssion

In [3]:
spark = SparkSession.builder.master("local[*]").appName("Sql 2019 Survey").getOrCreate()

In [5]:
df_2019_survey = spark.read.load("spark_files/survey_results_public.csv", format="csv", header=True, inferSchema = True)

Print Schema

In [6]:
df_2019_survey.printSchema()

root
 |-- Respondent: integer (nullable = true)
 |-- MainBranch: string (nullable = true)
 |-- Hobbyist: string (nullable = true)
 |-- OpenSourcer: string (nullable = true)
 |-- OpenSource: string (nullable = true)
 |-- Employment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Student: string (nullable = true)
 |-- EdLevel: string (nullable = true)
 |-- UndergradMajor: string (nullable = true)
 |-- EduOther: string (nullable = true)
 |-- OrgSize: string (nullable = true)
 |-- DevType: string (nullable = true)
 |-- YearsCode: string (nullable = true)
 |-- Age1stCode: string (nullable = true)
 |-- YearsCodePro: string (nullable = true)
 |-- CareerSat: string (nullable = true)
 |-- JobSat: string (nullable = true)
 |-- MgrIdiot: string (nullable = true)
 |-- MgrMoney: string (nullable = true)
 |-- MgrWant: string (nullable = true)
 |-- JobSeek: string (nullable = true)
 |-- LastHireDate: string (nullable = true)
 |-- LastInt: string (nullable = true)
 |-- FizzBuzz: 

In [8]:
len(df_2019_survey.columns)

85

In [10]:
df_2019_survey.first()

Row(Respondent=1, MainBranch='I am a student who is learning to code', Hobbyist='Yes', OpenSourcer='Never', OpenSource='The quality of OSS and closed source software is about the same', Employment='Not employed, and not looking for work', Country='United Kingdom', Student='No', EdLevel='Primary/elementary school', UndergradMajor='NA', EduOther='Taught yourself a new language, framework, or tool without taking a formal course', OrgSize='NA', DevType='NA', YearsCode='4', Age1stCode='10', YearsCodePro='NA', CareerSat='NA', JobSat='NA', MgrIdiot='NA', MgrMoney='NA', MgrWant='NA', JobSeek='NA', LastHireDate='NA', LastInt='NA', FizzBuzz='NA', JobFactors='NA', ResumeUpdate='NA', CurrencySymbol='NA', CurrencyDesc='NA', CompTotal='NA', CompFreq='NA', ConvertedComp='NA', WorkWeekHrs='NA', WorkPlan='NA', WorkChallenge='NA', WorkRemote='NA', WorkLoc='NA', ImpSyn='NA', CodeRev='NA', CodeRevHrs='NA', UnitTests='NA', PurchaseHow='NA', PurchaseWhat='NA', LanguageWorkedWith='HTML/CSS;Java;JavaScript;Py

#### 1. Average age by gender.

In [39]:
df_2019_survey=df_2019_survey.withColumn("Age", df_2019_survey.Age.cast(IntegerType()))


In [41]:
df_2019_survey.printSchema()

root
 |-- Respondent: integer (nullable = true)
 |-- MainBranch: string (nullable = true)
 |-- Hobbyist: string (nullable = true)
 |-- OpenSourcer: string (nullable = true)
 |-- OpenSource: string (nullable = true)
 |-- Employment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Student: string (nullable = true)
 |-- EdLevel: string (nullable = true)
 |-- UndergradMajor: string (nullable = true)
 |-- EduOther: string (nullable = true)
 |-- OrgSize: string (nullable = true)
 |-- DevType: string (nullable = true)
 |-- YearsCode: string (nullable = true)
 |-- Age1stCode: string (nullable = true)
 |-- YearsCodePro: string (nullable = true)
 |-- CareerSat: string (nullable = true)
 |-- JobSat: string (nullable = true)
 |-- MgrIdiot: string (nullable = true)
 |-- MgrMoney: string (nullable = true)
 |-- MgrWant: string (nullable = true)
 |-- JobSeek: string (nullable = true)
 |-- LastHireDate: string (nullable = true)
 |-- LastInt: string (nullable = true)
 |-- FizzBuzz: 

In [40]:
df_2019_survey.select('Gender','Age').groupBy('Gender').avg().show(8)

+--------------------+------------------+
|              Gender|          avg(Age)|
+--------------------+------------------+
|                 Man|30.424181874074282|
|                  NA| 30.35640785781104|
|Man;Non-binary, g...|28.908536585365855|
|           Woman;Man| 26.23170731707317|
|Woman;Man;Non-bin...| 29.78723404255319|
|Woman;Non-binary,...|28.210884353741495|
|Non-binary, gende...|29.018036072144287|
|               Woman| 29.49538679914833|
+--------------------+------------------+



#### 2. Top five countries by number of respondents.

In [68]:
df_2019_survey.select("Country", "Respondent").groupby("Country").agg(count("Respondent"). \
                      alias("Respondent_countrywise")).sort(desc("Respondent_countrywise")).show(5)
# new_df.sort(new_df['count'].desc()).show()


+--------------+----------------------+
|       Country|Respondent_countrywise|
+--------------+----------------------+
| United States|                 20949|
|         India|                  9061|
|       Germany|                  5866|
|United Kingdom|                  5737|
|        Canada|                  3395|
+--------------+----------------------+
only showing top 5 rows



In [89]:
df_2019_survey.select("Country", "Respondent").groupby("Country").count().sort(desc("count")).show(5)

+--------------+-----+
|       Country|count|
+--------------+-----+
| United States|20949|
|         India| 9061|
|       Germany| 5866|
|United Kingdom| 5737|
|        Canada| 3395|
+--------------+-----+
only showing top 5 rows



#### 3. The oldest age to write the first line of code or program for United States respondents with a computer   science, computer engineering, software engineering, mathematics, or statistics degree.

In [59]:
df_2019_survey = df_2019_survey.withColumn("Age1stCode", df_2019_survey.Age1stCode.cast(IntegerType()))

In [62]:
df_2019_survey.select("Country", "UndergradMajor", "Age1stCode").filter((df_2019_survey['Country'] == 'United States')\
      & ((df_2019_survey["UndergradMajor"] == "Computer science, computer engineering, or software engineering")\
         |(df_2019_survey["UndergradMajor"] == "Mathematics or statistics"))).sort(desc('Age1stCode')).show (100)

+-------------+--------------------+----------+
|      Country|      UndergradMajor|Age1stCode|
+-------------+--------------------+----------+
|United States|Mathematics or st...|        56|
|United States|Computer science,...|        54|
|United States|Computer science,...|        50|
|United States|Computer science,...|        50|
|United States|Computer science,...|        50|
|United States|Computer science,...|        49|
|United States|Computer science,...|        49|
|United States|Computer science,...|        48|
|United States|Computer science,...|        47|
|United States|Computer science,...|        46|
|United States|Computer science,...|        43|
|United States|Computer science,...|        42|
|United States|Computer science,...|        41|
|United States|Mathematics or st...|        41|
|United States|Computer science,...|        40|
|United States|Computer science,...|        40|
|United States|Computer science,...|        40|
|United States|Computer science,...|    

In [71]:
df_2019_survey.select("Country","UndergradMajor", "Age1stCode",).filter((df_2019_survey['Country'] == 'United States')\
      & ((df_2019_survey["UndergradMajor"] == "Computer science, computer engineering, or software engineering")\
         |(df_2019_survey["UndergradMajor"] == "Mathematics or statistics"))) \
    .groupby("Country").max("Age1stCode").show()

+-------------+---------------+
|      Country|max(Age1stCode)|
+-------------+---------------+
|United States|             56|
+-------------+---------------+



3. The oldest age to write the first line of code or program for United States respondents with a computer   
   science, computer engineering, software engineering, mathematics, or statistics degree.

In [67]:
df_2019_survey.select("Country", "UndergradMajor", "Age1stCode").filter((df_2019_survey['Country'] == 'United States')\
      & ((df_2019_survey["UndergradMajor"] == "Computer science, computer engineering, or software engineering")\
         |(df_2019_survey["UndergradMajor"] == "Mathematics or statistics"))) \
    .groupby("Country", "UndergradMajor").max("Age1stCode").show()

+-------------+--------------------+---------------+
|      Country|      UndergradMajor|max(Age1stCode)|
+-------------+--------------------+---------------+
|United States|Computer science,...|             54|
|United States|Mathematics or st...|             56|
+-------------+--------------------+---------------+

